In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

AttributeError: module 'numpy' has no attribute 'ndarray'

In [2]:
df = pd.read_csv("completeDataset.csv")

In [5]:
len(df)

1021221

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r1-target")
model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r1-target")

c:\Users\Alexandre\anaconda3\envs\tr10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
tokens = []

for a in df['Tweet Treated']: 
    tokens.append(tokenizer(a, padding=True, max_length=512, truncation=True, return_tensors="pt").to(device))

model = model.to(device)

In [6]:
inferences = []

with torch.no_grad():
    for tk in tokens:
        md = model(**tk)
        logits = md.logits
        scores = logits.softmax(dim=1).tolist()[0]
        predicted_class_id = logits.argmax().item()
        scores.insert(0 ,model.config.id2label[predicted_class_id])
        inferences.append(scores)

In [7]:
col_outpus =  ['model_inference', 'normal_score', 'hate_speech_score']
output_df  = pd.DataFrame(inferences,columns = col_outpus)
output_df.head()

,model_inference,normal_score,hate_speech_score
0,nothate,0.999554,0.000447
1,nothate,0.999679,0.000321
2,nothate,0.999711,0.000289
3,nothate,0.998655,0.001345
4,nothate,0.999884,0.000116


In [8]:
output_df.hate_speech_score = output_df.hate_speech_score.round(4)
output_df.normal_score = output_df.normal_score.round(4)

In [9]:
output_df = output_df.reset_index()

completeDataset = pd.concat([df, output_df], axis=1)

In [10]:
import csv
completeDataset.to_csv("completeDataset_inference.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)